# Model Development Example with MLFlow

This notebook serves as an example of how MLFlow can be used in the development of a machine learning model.

The rest of the notebook is organized as follows
- 1) The Problem;
- 2) Setup Development Environment;
- 3) Training different Models;
- 4) Setting the Best Model to Production and the Second best to stage (so that we can deploy an API later).

In [1]:
!which python

/media/vinicius/Dados/poetry/virtualenvs/ml-pipes-VBbH4xSK-py3.8/bin/python


In [2]:
# Import Libs
import os
from dotenv import load_dotenv
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

from dao.CreditCardDefault import load_creditcard_dataset
from trainers.h2o_automl import H2OClassifier
from trainers.pycaret import PycaretClassifier
from trainers.spark import SparkClassifier

load_dotenv(dotenv_path='../ml-pipes/')
pd.set_option('display.max_columns', 500)

EXPERIMENT_NAME = "CreditCardDefault"
TRACKING_URI = 'http://127.0.0.1:5000'
CREDIT_CARD_MODEL_NAME = EXPERIMENT_NAME
THRESHOLD = 0.5
CHAMPION_METRIC = 'ks'
FOLDS = 5

## 1) The problem

The first thing we need to have to build a model is a problem to solve. Here it is used as example the [Credit Card Default from Kagle](https://www.kaggle.com/mlg-ulb/creditcardfraud), where basically the objective if to predict based on a few features whether or not a client will default on its credit card. The taret variable can assume the values 1, for default, and 0 for non default. Therefore it is a binary classification problem.

Bellow the dataset is imported and the first rows of the dataset. Note that the Time column has been removed from the original dataset.

Make sure that you have the file `example-model-development/dao/data/creditcard.csv` inside your project.

In [4]:
dataset = load_creditcard_dataset()
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
130658,1.368404,-0.423220,0.410756,-0.806396,-0.642184,-0.331725,-0.523129,-0.115005,-0.966748,0.520532,-0.042983,0.166267,1.618911,-0.420917,0.942149,0.984929,0.285962,-2.080199,0.372764,0.160176,-0.068704,-0.240810,0.094137,-0.261473,0.248152,-0.435464,0.033116,0.020626,20.00,0
18988,1.357532,-1.303938,-0.199479,-1.498897,-1.127011,-0.272185,-0.860090,0.051638,-1.996670,1.629421,0.798558,-1.351043,-1.714358,0.500234,0.174627,-0.537946,0.645634,0.226221,-0.105278,-0.353391,-0.133541,-0.231315,-0.082597,-0.361752,0.407616,-0.112577,-0.004237,0.001871,77.20,0
110902,-1.405474,0.994269,1.467485,-1.986441,-1.243961,-0.523304,-0.314165,0.422654,2.200189,-0.476182,-0.092892,0.883387,-0.059324,-0.587641,0.944823,-1.115766,0.518518,-0.530520,0.432153,0.404095,-0.080130,0.358614,-0.034897,0.445072,0.039238,-0.075287,0.222351,-0.013392,0.99,0
104426,-7.550169,6.433746,-2.810013,-3.131537,-1.685287,-0.665016,-0.666020,1.505854,4.864181,7.834397,0.650600,1.399985,1.165278,-1.454507,0.133876,0.773779,-1.296268,0.023486,-0.904458,3.584708,-0.906316,-0.867041,0.276942,-0.896160,1.362156,0.823835,2.102968,0.818651,0.92,0
82413,-2.037267,1.941805,0.136512,-0.798728,-0.845243,-0.590195,-0.462000,1.425195,-0.348145,-0.542304,0.273056,1.158715,-0.052899,0.889831,-1.054842,0.609319,-0.168637,0.107530,0.116411,-0.041282,-0.096690,-0.410116,0.039041,0.022263,-0.032636,0.311809,0.131647,0.099978,1.00,0


## 2) Setup Development Environment

Now that a problem has been stated and some data to help solving the problem has been gathered, the next step is to setup our environment to make use of the mlflow tracking module. In order to do that we need to (i) make sure that our enviroment (the python session that is running this notebook) has acces to the bucket that we created and (ii) Setup an MLFlow Experiment.

In [5]:
# Setting credentials to bucket (here is harder  coded for )
# Here you set the credentials created in the .env file
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://127.0.0.1:9000'
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('MINIO_ACCESS_KEY')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('MINIO_SECRET_KEY')

**MLFlow is built upon the concept of experiments. A experiment is a series of fits, where parameters, metrics, models and artifacts can be associated with the respective fit (in an machine learning package agnostic way).**

The code bellow tries to create an experiment, if that experiments already existis then it sets the experiment to the active one.

In [6]:
mlflow.set_tracking_uri(TRACKING_URI)
try:
    experiment = mlflow.create_experiment(EXPERIMENT_NAME)
except Exception:
    client = MlflowClient()
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

mlflow.set_experiment(EXPERIMENT_NAME)

## 3) Training different Models

The next step if to train, evaluate and log a few different models. In order to demonstrate that MLFlow allows us to use different machine learning packages we will train an H2O autoML, SkLearn models (using pycaret) and spark. Now is the time where MLFlow is put into action: For each model that if fitted it will be logged a few parameters, metrics, artifacts and the models it self. To understand how this is done it checkout the classifiers definitions in the `./trainers` folder and the [MLFlow Logging Documentaion](https://www.mlflow.org/docs/latest/tracking.html#logging-data-to-runs), it all happens inside the `mlflow.start_run()` context manager. 

In [7]:
H2OClassifier(
    run_name='H2O',
    max_mem_size='3G',
    threshold=THRESHOLD,
    df=dataset,
    target_col='Class',
    sort_metric='aucpr',
    max_models=8,
    max_runtime_secs=10,
    nfolds=FOLDS,
    seed=90
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /media/vinicius/Dados/poetry/virtualenvs/ml-pipes-VBbH4xSK-py3.8/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqa4jhzqd
  JVM stdout: /tmp/tmpqa4jhzqd/h2o_vinicius_started_from_python.out
  JVM stderr: /tmp/tmpqa4jhzqd/h2o_vinicius_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,1 month and 25 days
H2O_cluster_name:,H2O_from_python_vinicius_puf7c7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |
09:19:32.277: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%
Could not find exact threshold 0.5; using closest threshold found 0.25716614702173335.
Could not find exact threshold 0.5; using closest threshold found 0.25716614702173335.
Could not find exact threshold 0.5; using closest threshold found 0.25716614702173335.
Could not find exact threshold 0.5; using closest threshold found 0.25716614702173335.
Could not find exact threshold 0.5; using closest threshold found 0.25716614702173335.


In [9]:
PycaretClassifier(
        experiment_name=EXPERIMENT_NAME,
        run_name='Pycaret2',
        sort_metric='precision',
        df=dataset,
        target='Class',
        threshold=THRESHOLD,
        n_best_models=3,
        data_split_stratify=True,
        nfolds=FOLDS,
        normalize=True,
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=True,
        multicollinearity_threshold=0.95,
        session_id=54321
)

,Parameters
n_components,None
priors,None
shrinkage,None
solver,svd
store_covariance,False
tol,0.0001


Error in logging parameter for                                 pycaret_precision_2
[Errno 2] No such file or directory: 'Hyperparameters.png'


In [10]:
SparkClassifier(
    df = dataset,
    target_col = 'Class',
    run_name = 'spark_classifier',
    max_mem_size = 4,
    n_cores = 4,
    seed = 90
)

If everything runned as expected you can now check the MLFlow Server at [http://127.0.0.1:5000](http://127.0.0.1:5000) to compare and explore the models runs.

## 4) Setting the Best Model to Production

The final step in this notebook if to set to production the model with the best selected metric, imported as `CHAMPION_METRIC`. This is done to show is is possible to create an automated workflow using MLFlow to deplot a model. However it is also possible to deplot the model using the [UI server](https://www.mlflow.org/docs/latest/model-registry.html#ui-workflow).

Once this is done you can return to the README file to check how the model is now deployed.

In [10]:
# Getting The best Model according to CHAMPION_METRIC
champion = MlflowClient().search_runs(
    experiment_ids=[
        str(
            mlflow.get_experiment_by_name(name=EXPERIMENT_NAME).experiment_id
        )
    ],
    run_view_type=ViewType.ALL,
    order_by=[f"metrics.{CHAMPION_METRIC} DESC"],
    max_results=1
)
run_id = champion[0].info.run_id

# Registering Model in model registery
model = mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name=CREDIT_CARD_MODEL_NAME
)

# Setting version 1
MlflowClient().update_model_version(
    name=CREDIT_CARD_MODEL_NAME,
    version=model.version,
    description='Deploying model with model registery'
)

# Setting it to production
MlflowClient().transition_model_version_stage(
    name=CREDIT_CARD_MODEL_NAME,
    version=model.version,
    stage="Staging"
)

Registered model 'CreditCardDefault' already exists. Creating a new version of this model...
2021/05/19 09:02:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CreditCardDefault, version 3
Created version '3' of model 'CreditCardDefault'.


<ModelVersion: creation_timestamp=1621425758707, current_stage='Staging', description='Deploying model with model registery', last_updated_timestamp=1621425758825, name='CreditCardDefault', run_id='c325b227a2d24a3994ca8e75b0201117', run_link='', source='/media/vinicius/Dados/projects/ml-pipes/mlflow_artifact_store/1/c325b227a2d24a3994ca8e75b0201117/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>